# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import optimize

In [2]:
# Define data for this example

x1 = [0, 0, 1, 0, 1, 1, 0]
x2 = [0, 1, 0, 1, 0, 1, 0]
x3 = [1, 1, 1, 0, 0, 1, 0]
y = [0, 1, 1, 1, 1, 0, 0]

In [3]:
# Combine x lists and one column vector of zeroes to make inputs matrix
# The vector of zeroes represents bias

inputs = np.array(list(zip(x1, x2, x3, np.ones(len(x1)))))
inputs

array([[0., 0., 1., 1.],
       [0., 1., 1., 1.],
       [1., 0., 1., 1.],
       [0., 1., 0., 1.],
       [1., 0., 0., 1.],
       [1., 1., 1., 1.],
       [0., 0., 0., 1.]])

In [4]:
# The y list represents correct output; create a correct output vector

correct_output = np.array([[val] for val in y])
correct_output.shape

(7, 1)

In [5]:
# Create specified perceptron; credit to Welch Labs and RA/LSDS
# Note that this is a supervised learning problem, given the available input data


class MultilayerPerceptron():
    def __init__(self):
        '''
        Define node size of input, hidden layer (one hidden layer only here), and output layer;
        plus weights (two). These values are all fixed
        '''
        self.input_size = 3 + 1  # includes column of ones
        self.hidden_layer_size = 4
        self.output_layer_size = 1
        
        # Weights (parameters)
        self.L1_weights = np.random.randn(self.input_size, self.hidden_layer_size)  # WL calls self.W1
        self.L2_weights = np.random.randn(self.hidden_layer_size, self.output_layer_size)  # WL calls self.W2
    
    def forward(self, X):
        '''Propagate inputs forward through network'''
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)  # WL calls this self.z2; summation is the idea
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)  # WL calls this self.a2
        # Weighted sum between hidden layer and output layer
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)  # WL calls this self.z3
        y_hat = self.sigmoid(self.output_sum)  # called y_hat because is an estimate of output data 
        return y_hat
    
    def sigmoid(self, s):
        '''Apply sigmoid activation function to scalar, vector, or matrix'''
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        '''Calculate gradient of sigmoid'''
        return np.exp(-s) / ((1 + np.exp(-s))**2)
            
    def cost_function(self, X, y):
        '''
        Compute cost for given X, y, using weights already stored in class. Cost is a
        measure of how incorrect model is after at least one complete forward propagation
        '''
        self.y_hat = self.forward(X)
        J = 0.5 * sum((y - self.y_hat)**2)  # J is term for cost output unit
        return J
        
    def cost_function_prime(self, X, y):
        '''Compute derivative with respect to L1_weights and L2_weights for a given X and y'''
        self.y_hat = self.forward(X)
        
        delta3 = np.multiply(-(y - self.y_hat), self.sigmoid_prime(self.output_sum))
        dJdL2 = np.dot(self.activated_hidden.T, delta3)
        
        delta2 = np.dot(delta3, self.L2_weights.T) * self.sigmoid_prime(self.hidden_sum)
        dJdL1 = np.dot(X.T, delta2)  
        
        return dJdL1, dJdL2
    
    # Helper Functions for interacting with other classes
    def get_params(self):
        '''Get L1_weights and L2_weights unrolled into vector'''
        params = np.concatenate((self.L1_weights.ravel(), self.L2_weights.ravel()))
        return params
    
    def set_params(self, params):
        '''Set L1 and L2 using single parameter vector'''
        L1_start = 0
        L1_end = self.hidden_layer_size * self.input_size
        self.L1_weights = np.reshape(params[L1_start: L1_end], (self.input_size, self.hidden_layer_size))
        L2_end = L1_end + self.hidden_layer_size * self.output_layer_size
        self.L2_weights = np.reshape(params[L1_end: L2_end], (self.hidden_layer_size, self.output_layer_size))
    
    def compute_gradients(self, X, y):
        '''
        Returns the vector that takes us in the most downward direction along some function
        in hyperspace that has as many dimensions as we have weights--2, in this case
        '''
        dJdL1, dJdL2 = self.cost_function_prime(X, y)
        return np.concatenate((dJdL1.ravel(), dJdL2.ravel()))

In [6]:
class trainer():
    def __init__(self, N):
        # Make Local reference to network
        self.N = N
    
    def callback_func(self, params):
        self.N.set_params(params)
        self.J.append(self.N.cost_function(self.X, self.y))   
        
    def cost_function_wrapper(self, params, X, y):
        self.N.set_params(params)
        cost = self.N.cost_function(X, y)
        grad = self.N.compute_gradients(X,y)
        
        return cost, grad
        
    def train(self, X, y):
        # Make an internal variable for the callback function
        self.X = X
        self.y = y

        # Make empty list to store costs
        self.J = []
        
        params0 = self.N.get_params()

        options = {'maxiter': 200, 'disp' : True}
        _res = optimize.minimize(self.cost_function_wrapper, params0, jac=True, method='BFGS', \
                                 args=(X, y), options=options, callback=self.callback_func)

        self.N.set_params(_res.x)
        self.optimization_results = _res

In [7]:
MP = MultilayerPerceptron()

In [8]:
T = trainer(MP)

In [9]:
T.train(inputs, correct_output)

Optimization terminated successfully.
         Current function value: 0.000000
         Iterations: 47
         Function evaluations: 53
         Gradient evaluations: 53


In [10]:
print("Correct output: \n", correct_output)
print("Predicted Output: \n" + str(MP.forward(inputs))) 
print("Loss: \n" + str(np.mean(np.square(y - MP.forward(inputs)))))  # mean sum squared loss

Correct output: 
 [[0]
 [1]
 [1]
 [1]
 [1]
 [0]
 [0]]
Predicted Output: 
[[1.06679013e-16]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.00000000e+00]
 [1.85108641e-06]
 [7.10327890e-04]]
Loss: 
0.4896797163300973


## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [11]:
# See http://rasbt.github.io/mlxtend/user_guide/data/mnist_data/ 
from mlxtend.data import mnist_data

In [12]:
# Create X, y matrices

X, y = mnist_data()

In [13]:
# Normalize values in X for rgb(0, 255)

X = X / 255

In [14]:
# Inspect data

# print('Dimensions: {} x {}'.format(X.shape[0], X.shape[1]), '\n')
# print('First 140 values in 0th row of X: \n\n', X[0][:140])
X.shape

(5000, 784)

In [15]:
# Replace y with a vector of probabilities that the digit is what it is - credit to DMA/LSDS01
y_expanded = []
for idx, num in enumerate(y):
    row = np.zeros(10)
    row[num] = 1
    y_expanded.append(row)

y = np.array(y_expanded)
y.shape

(5000, 10)

In [29]:
# Build MLP - based, in part, on 3blue1brown [Sanderson] description at 
# https://www.youtube.com/watch?v=aircAruvnKk


class MNistMultilayerPerceptron():
    def __init__(self):
        '''
        Define node count (size) for input data, each hidden layer, and output layer;
        plus weights (three). These values are all fixed
        '''
        self.input_size = 784
        self.hidden_layer1_size = 16
        self.hidden_layer2_size = 16
        self.output_layer_size = 10
        
        # Weights (parameters)
        self.L1_weights = np.random.randn(self.input_size, self.hidden_layer1_size)
        self.L2_weights = np.random.randn(self.hidden_layer1_size, self.hidden_layer2_size)
        self.L3_weights = np.random.randn(self.hidden_layer2_size, self.output_layer_size)
    
    def forward(self, X):
        '''Propagate inputs forward through network'''
        # Weighted sum between inputs and hidden layer
        self.hidden_sum1 = np.dot(X, self.L1_weights)
        # Activations of weighted sum 1
        self.activated_hidden1 = self.sigmoid(self.hidden_sum1)
        # Weighted sum between hidden layer 1 and hidden layer 2
        self.hidden_sum2 = np.dot(self.activated_hidden1, self.L2_weights)
        # Activations of weighted sum 2
        self.activated_hidden2 = self.sigmoid(self.hidden_sum2)
        # Weighted sum between hidden layer 2 and output layer
        self.output_sum = np.dot(self.activated_hidden2, self.L3_weights)
        y_hat = self.sigmoid(self.output_sum)
        # print('When forward(self, X) is run, self.output_sum is:', self.output_sum)
        return y_hat
    
    def relu(self, s):
        '''Apply ReLU activation function'''
        return np.maximum(s, 0)
    
    def relu_prime(self, s):
        '''The derivative of ReLU is equivalent to the slope of the ReLU graph'''
        # see https://kawahara.ca/what-is-the-derivative-of-relu/
        if int(s) > 0: return 1
        else: return 0
    
    def sigmoid(self, s):
        '''Apply sigmoid activation function to scalar, vector, or matrix'''
        return 1 / (1 + np.exp(-s))
    
    def sigmoid_prime(self, s):
        '''Calculate gradient of sigmoid'''
        return np.exp(-s) / ((1 + np.exp(-s))**2)
    
    def cost_function(self, X, y):
        '''
        Compute cost for given X, y, using weights already stored in class. Cost is a
        measure of how incorrect model is after at least one complete forward propagation
        '''
        self.y_hat = self.forward(X)
        J = 0.5 * sum((y - self.y_hat)**2)  # J is term for cost output unit
        return J
    
    def cost_function_prime(self, X, y):
        '''Compute derivative with respect to L1_weights, L2_weights, and L3_weights for a given X and y'''
        self.y_hat = self.forward(X)
        
        delta4 = np.multiply(-(y - self.y_hat), self.sigmoid_prime(self.output_sum))
        dJdL3 = np.dot(self.activated_hidden2.T, delta4)
        
        delta3 = np.dot(delta4, self.L3_weights.T) * self.sigmoid_prime(self.hidden_sum2)
        dJdL2 = np.dot(self.activated_hidden1.T, delta3)
        
        delta2 = np.dot(delta3, self.L2_weights.T) * self.sigmoid_prime(self.hidden_sum1)
        dJdL1 = np.dot(X.T, delta2)  
        
        return dJdL1, dJdL2, dJdL3
    
    # Helper Functions for interacting with other classes
    def get_params(self):
        '''Get L1_weights, L2_weights, and L3_weights unrolled into vector'''
        params = np.concatenate((self.L1_weights.ravel(), self.L2_weights.ravel(), self.L3_weights.ravel()))
        return params
    
    def set_params(self, params):
        '''Set L1, L2, and L3 using single parameter vector'''
        L1_start = 0
        L1_end = self.hidden_layer1_size * self.input_size
        self.L1_weights = np.reshape(params[L1_start: L1_end], (self.input_size, self.hidden_layer1_size))
        L2_end = L1_end + self.hidden_layer1_size * self.hidden_layer2_size
        self.L2_weights = np.reshape(params[L1_end: L2_end], (self.hidden_layer1_size, self.hidden_layer2_size))
        L3_end = L2_end + self.hidden_layer2_size * self.output_layer_size
        self.L3_weights = np.reshape(params[L2_end: L3_end], (self.hidden_layer2_size, self.output_layer_size))
    
    def compute_gradients(self, X, y):
        '''
        Returns the vector that takes us in the most downward direction along some function
        in hyperspace that has as many dimensions as we have weights--2, in this case
        '''
        dJdL1, dJdL2, dJdL3 = self.cost_function_prime(X, y)
        return np.concatenate((dJdL1.ravel(), dJdL2.ravel(), dJdL3.ravel()))

In [30]:
MNMP = MNistMultilayerPerceptron()

In [31]:
T = trainer(MNMP)

In [32]:
T.train(X, y)

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [33]:
print("Correct output: \n", y)
print("Predicted Output: \n" + str(MNMP.forward(X))) 
print("Loss: \n" + str(np.mean(np.square(y - MNMP.forward(X)))))  # mean sum squared loss

Correct output: 
 [[1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 [1. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]
 [0. 0. 0. ... 0. 0. 1.]]
Predicted Output: 
[[0.12428342 0.15411933 0.22286017 ... 0.832137   0.14956555 0.86184676]
 [0.11826002 0.08835938 0.18060253 ... 0.80371118 0.12031481 0.8609318 ]
 [0.21700312 0.11230913 0.28483903 ... 0.63891409 0.07989932 0.77786567]
 ...
 [0.18607539 0.07714101 0.75373593 ... 0.40616769 0.1026493  0.79189352]
 [0.14717859 0.06608888 0.4617302  ... 0.70407657 0.10759739 0.755218  ]
 [0.24524085 0.07316109 0.29127684 ... 0.64464306 0.14675023 0.8156784 ]]
Loss: 
0.30263469146267535


## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?